In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import swifter
import random

In [3]:
import pandas
import random

n = 148075238 #number of records in file
s = 10000 #desired sample size
# filename = "data.txt"
skip = sorted(random.sample(range(n),n-s))
# df = pandas.read_csv(filename, skiprows=skip)

In [4]:
def ep_to_day(ep):
    return datetime.fromtimestamp(ep/1000).strftime("%A")

In [5]:
all_features = ["text_ tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
                "tweet_type","language", "tweet_timestamp", "enaged_with_user_id", "engaged_with_user_follower_count",\
               "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "enaging_user_id", "enaging_user_follower_count", "enaging_user_following_count", "enaging_user_is_verified",\
               "enaging_user_account_creation", "engagee_follows_engager", "reply_timestamp", "retweet_timestamp",\
                "retweet_with_comment_timestamp", "like_timestamp"]
training = pd.read_csv('s3://recsys-challenge-2020/training.tsv', encoding="utf-8", names = all_features, sep="\x01", skiprows=skip)

In [6]:
training['day'] = training.apply(lambda x: ep_to_day(x['tweet_timestamp']), axis = 1)

In [7]:
training

,text_ tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,enaged_with_user_id,...,enaging_user_follower_count,enaging_user_following_count,enaging_user_is_verified,enaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp,day
0,101\t37312\t10107\t12415\t10689\t10309\t42374\...,NaN,C39511002DCF7995F06B3A1FDCD4A73C,Video,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581206643,39D81F938CDE8F617D791C9B4A976F25,...,121,323,False,1309534379,False,NaN,NaN,NaN,1.581222e+09,Monday
1,101\t11590\t17835\t131\t10142\t23433\t69191\t2...,NaN,95E1C1E253ED1F31D61529AF0CDACA2D,Photo,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581014347,F2B4600F51F072136C61E61DDBD1DC6D,...,199,453,False,1315227442,False,NaN,NaN,NaN,1.581031e+09,Monday
2,101\t12489\t13028\t10149\t10345\t16938\t112\t1...,NaN,A418E905CC2A24143F56F75C787FFF2D,Photo,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581231514,03B7DFFA03777B2CC2B4807E46BFEA9F,...,133,965,False,1498560017,False,NaN,1.581289e+09,NaN,1.581289e+09,Monday
3,101\t14864\t41878\t10635\t106\t49983\t10911\t4...,NaN,E19388EA917BEEDD2AA98D247686AE0E,Photo,36E38586E1045FA2F7581BD73B968E34\t02DEDAF77CD7...,E83F57233C20A926F4963EC6348D19AD\t892B6E0BC037...,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581042609,E6E001C1A6B35EA379FEA2F1ECC38042,...,1122,207,False,1550976224,False,NaN,NaN,NaN,NaN,Monday
4,101\t177\t100\t181\t10472\t34875\t10491\t25585...,NaN,636BD83AD1EA7B27EAD776FD8F0B5FD0,NaN,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581282209,8F21C9E121C7253B1B0372021492FBE9,...,132,151,False,1330987315,True,NaN,NaN,NaN,1.581285e+09,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,101\t100\t117\t10192\t27904\t10940\t11951\t103...,C35FABC5BDBFFC4D9746C3CB3973FDC0\t20853445F27D...,70CAF62CD9781AA3A909DEFFBC23634D,Photo\tPhoto\tPhoto,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581411829,076D9DBAA8246A2C23BF1C06F0299CD1,...,359,503,False,1567389600,False,NaN,1.581413e+09,NaN,1.581413e+09,Monday
9996,101\t56898\t137\t11529\t11643\t50241\t11790\t1...,NaN,E480BA0BB5AE1DBF3F0FAB11793FE4C5,NaN,NaN,NaN,Retweet,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581323918,7E978D8FC86B9CC43F6BF1A2DA3480DF,...,110,365,False,1521962395,False,NaN,NaN,NaN,NaN,Monday
9997,101\t107\t11045\t69532\t263\t31481\t117\t10126...,NaN,8DE171558296EC3DCB6661CCA626A6EE,Video,NaN,NaN,TopLevel,ECED8A16BE2A5E8871FD55F4842F16B1,1581199241,40D3A3AC6E43BF0C0F88A284149F1DA4,...,398,620,False,1550682299,False,NaN,NaN,NaN,1.581201e+09,Monday
9998,101\t47271\t11447\t12022\t80932\t11517\t141\t1...,NaN,94DE28F27BAC3564CBD280B2CB470331,NaN,NaN,NaN,TopLevel,ECED8A16BE2A5E8871FD55F4842F16B1,1581316580,4691B1BDBBA890C421CF75CA5BE2333A,...,1028,2101,False,1468369627,True,NaN,NaN,NaN,1.581318e+09,Monday


In [8]:
training['retweet_with_comment_bool'] = training.retweet_with_comment_timestamp.fillna(0)
training.loc[training.retweet_with_comment_bool != 0.0, 'retweet_with_comment_bool'] = 1.0

In [9]:
training['present_links_bool'] = training.present_links.fillna(0)
training.loc[training.present_links_bool != 0, 'present_links_bool'] = 1

In [10]:
(training.retweet_with_comment_bool == 1.0).sum()

85

In [11]:
training.columns

Index(['text_ tokens', 'hashtags', 'tweet_id', 'present_media',
       'present_links', 'present_domains', 'tweet_type', 'language',
       'tweet_timestamp', 'enaged_with_user_id',
       'engaged_with_user_follower_count', 'engaged_with_user_following_count',
       'engaged_with_user_is_verified', 'engaged_with_user_account_creation',
       'enaging_user_id', 'enaging_user_follower_count',
       'enaging_user_following_count', 'enaging_user_is_verified',
       'enaging_user_account_creation', 'engagee_follows_engager',
       'reply_timestamp', 'retweet_timestamp',
       'retweet_with_comment_timestamp', 'like_timestamp', 'day',
       'retweet_with_comment_bool', 'present_links_bool'],
      dtype='object')

In [13]:
training['day'].nunique()

1

In [14]:
training_subset = training[['present_media', 'present_links', 'present_domains', 'tweet_type', 'language',\
                     'enaged_with_user_id', 'engaged_with_user_is_verified', 'enaging_user_id',\
                    'enaging_user_is_verified', 'engagee_follows_engager', 'present_links_bool', 'retweet_with_comment_bool']]

In [15]:
training_subset

,present_media,present_links,present_domains,tweet_type,language,enaged_with_user_id,engaged_with_user_is_verified,enaging_user_id,enaging_user_is_verified,engagee_follows_engager,present_links_bool,retweet_with_comment_bool
0,Video,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,39D81F938CDE8F617D791C9B4A976F25,True,001F18E6A1BD8E7BD81702A2CAB59428,False,False,0,0.0
1,Photo,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,F2B4600F51F072136C61E61DDBD1DC6D,False,004A58665869FDB0335A7B35D8BE733E,False,False,0,0.0
2,Photo,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,03B7DFFA03777B2CC2B4807E46BFEA9F,True,005DECFBFD8573E52985B453E7B2F077,False,False,0,0.0
3,Photo,36E38586E1045FA2F7581BD73B968E34\t02DEDAF77CD7...,E83F57233C20A926F4963EC6348D19AD\t892B6E0BC037...,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,E6E001C1A6B35EA379FEA2F1ECC38042,False,0092889F27583B18D3AA0CFDD193C8AE,False,False,1,0.0
4,NaN,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,8F21C9E121C7253B1B0372021492FBE9,False,009539E64C2364766CD30FBFDA7BFBAC,False,True,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Photo\tPhoto\tPhoto,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,076D9DBAA8246A2C23BF1C06F0299CD1,False,FFADA91B3E8D8A7D24E6EC656C97AF1B,False,False,0,0.0
9996,NaN,NaN,NaN,Retweet,D3164C7FBCF2565DDF915B1B3AEFB1DC,7E978D8FC86B9CC43F6BF1A2DA3480DF,False,FFB1B77DF18DE60F44043BDD5A27575B,False,False,0,0.0
9997,Video,NaN,NaN,TopLevel,ECED8A16BE2A5E8871FD55F4842F16B1,40D3A3AC6E43BF0C0F88A284149F1DA4,False,FFCBEC436F828FD3BAC0E86E882FD58F,False,False,0,0.0
9998,NaN,NaN,NaN,TopLevel,ECED8A16BE2A5E8871FD55F4842F16B1,4691B1BDBBA890C421CF75CA5BE2333A,False,FFDC5F9CC056459919C123F028A5884A,False,True,0,0.0


In [16]:
X = training_subset.drop(['retweet_with_comment_bool'], axis = 1)
y = training_subset[['retweet_with_comment_bool']]

In [17]:
X.present_links_bool.nunique()

2

In [18]:
X

,present_media,present_links,present_domains,tweet_type,language,enaged_with_user_id,engaged_with_user_is_verified,enaging_user_id,enaging_user_is_verified,engagee_follows_engager,present_links_bool
0,Video,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,39D81F938CDE8F617D791C9B4A976F25,True,001F18E6A1BD8E7BD81702A2CAB59428,False,False,0
1,Photo,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,F2B4600F51F072136C61E61DDBD1DC6D,False,004A58665869FDB0335A7B35D8BE733E,False,False,0
2,Photo,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,03B7DFFA03777B2CC2B4807E46BFEA9F,True,005DECFBFD8573E52985B453E7B2F077,False,False,0
3,Photo,36E38586E1045FA2F7581BD73B968E34\t02DEDAF77CD7...,E83F57233C20A926F4963EC6348D19AD\t892B6E0BC037...,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,E6E001C1A6B35EA379FEA2F1ECC38042,False,0092889F27583B18D3AA0CFDD193C8AE,False,False,1
4,NaN,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,8F21C9E121C7253B1B0372021492FBE9,False,009539E64C2364766CD30FBFDA7BFBAC,False,True,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,Photo\tPhoto\tPhoto,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,076D9DBAA8246A2C23BF1C06F0299CD1,False,FFADA91B3E8D8A7D24E6EC656C97AF1B,False,False,0
9996,NaN,NaN,NaN,Retweet,D3164C7FBCF2565DDF915B1B3AEFB1DC,7E978D8FC86B9CC43F6BF1A2DA3480DF,False,FFB1B77DF18DE60F44043BDD5A27575B,False,False,0
9997,Video,NaN,NaN,TopLevel,ECED8A16BE2A5E8871FD55F4842F16B1,40D3A3AC6E43BF0C0F88A284149F1DA4,False,FFCBEC436F828FD3BAC0E86E882FD58F,False,False,0
9998,NaN,NaN,NaN,TopLevel,ECED8A16BE2A5E8871FD55F4842F16B1,4691B1BDBBA890C421CF75CA5BE2333A,False,FFDC5F9CC056459919C123F028A5884A,False,True,0


In [19]:
y

,retweet_with_comment_bool
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
9995,0.0
9996,0.0
9997,0.0
9998,0.0


In [20]:
X = X.fillna('unknown')

In [21]:
from collections import defaultdict
d = defaultdict(LabelEncoder)
fit = X.apply(lambda x: d[x.name].fit_transform(x))

In [22]:
fit

,present_media,present_links,present_domains,tweet_type,language,enaged_with_user_id,engaged_with_user_is_verified,enaging_user_id,enaging_user_is_verified,engagee_follows_engager,present_links_bool
0,5,1327,690,2,40,2118,1,5,0,0,0
1,1,1327,690,2,40,8854,0,13,0,0,0
2,1,1327,690,2,40,136,1,16,0,0,0
3,1,283,626,2,40,8362,0,23,0,0,1
4,6,1327,690,2,40,5161,0,25,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,3,1327,690,2,40,283,0,9982,0,0,0
9996,6,1327,690,1,40,4574,0,9983,0,0,0
9997,5,1327,690,2,44,2382,0,9988,0,0,0
9998,6,1327,690,2,44,2565,0,9990,0,1,0


In [23]:
y

,retweet_with_comment_bool
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
9995,0.0
9996,0.0
9997,0.0
9998,0.0


In [24]:
fs = SelectKBest(score_func=chi2, k='all')
fs.fit(fit, y)

SelectKBest(k='all', score_func=<function chi2 at 0x7f8ea0998950>)

In [25]:
for i in range(len(fs.scores_)):
	print('Feature %d: %f' % (i, fs.scores_[i]))
# plot the scores
# pyplot.bar([fit[i] for i in range(len(fs.scores_))], fs.scores_)
# pyplot.show()

Feature 0: 0.018381
Feature 1: 64.890742
Feature 2: 44.807987
Feature 3: 0.255971
Feature 4: 40.807773
Feature 5: 979.179695
Feature 6: 2.991647
Feature 7: 313.784918
Feature 8: 0.111447
Feature 9: 10.801263
Feature 10: 1.008939


In [22]:
fit.columns[5]

'enaged_with_user_id'

In [22]:
fit.columns[7]

'enaging_user_id'

In [29]:
fit.columns[10]

'present_links_bool'